In [86]:
import numpy as np
import pandas as pd
import h2o

h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,28 mins 02 secs
H2O cluster timezone:,America/Los_Angeles
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.2
H2O cluster version age:,8 months and 26 days !!!
H2O cluster name:,H2O_from_python_andrewsang_y5mxt5
H2O cluster total nodes:,1
H2O cluster free memory:,1.618 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


In [10]:
wine_df = h2o.import_file('winequality-red.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [74]:
wine_df.columns = [c.replace(' ','_') for c in wine_df.columns]

In [84]:
train, test, valid = wine_df.split_frame(ratios=([.8,.1]), seed = 1234)

In [134]:
param = {
    "ntrees" : 250, 
    "max_depth" : 6, 
    "learn_rate" : 0.1,
    "sample_rate" : 0.8, 
    "col_sample_rate_per_tree" : 0.8, 
    "min_rows" : 25,
    "seed": 1234, 
    "score_tree_interval": 100
}

from h2o.estimators import H2OXGBoostEstimator
model = H2OXGBoostEstimator(**param)
# X = wine_df.columns[:-1]
X = ['alcohol','sulphates','volatile_acidity','total_sulfur_dioxide']
model.train(x = X, 
            y = 'quality', 
            training_frame = train, 
            validation_frame = test)
model

xgboost Model Build progress: |███████████████████████████████████████████| 100%
Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_model_python_1578855675432_40


ModelMetricsRegression: xgboost
** Reported on train data. **

MSE: 0.15218037306927676
RMSE: 0.39010302878762265
MAE: 0.29625229102484185
RMSLE: 0.06060146605390641
Mean Residual Deviance: 0.15218037306927676

ModelMetricsRegression: xgboost
** Reported on validation data. **

MSE: 0.5371605618119828
RMSE: 0.7329123834483784
MAE: 0.5494922148553949
RMSLE: 0.11720938511656052
Mean Residual Deviance: 0.5371605618119828
Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance,validation_rmse,validation_mae,validation_deviance
,2020-01-12 11:49:33,0.011 sec,0.0,5.2000358,5.1381988,27.0403727,5.2452986,5.1644737,27.5131579
,2020-01-12 11:49:33,0.123 sec,100.0,0.4822792,0.3709728,0.2325932,0.7241212,0.5501849,0.5243515
,2020-01-12 11:49:33,0.218 sec,200.0,0.4152928,0.3166449,0.1724681,0.7301124,0.5513785,0.5330642
,2020-01-12 11:49:33,0.292 sec,250.0,0.3901030,0.2962523,0.1521804,0.7329124,0.5494922,0.5371606


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
alcohol,960.7824097,1.0,0.3499666
volatile_acidity,634.3043823,0.6601957,0.2310465
total_sulfur_dioxide,576.5860596,0.6001214,0.2100225
sulphates,573.6813965,0.5970981,0.2089644


In [135]:
wine_df.head(1)

fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
7.4,0.7,0,1.9,0.076,11,34,0.9978,3.51,0.56,9.4,5


In [137]:
# pred_dict = {'alcohol': 9, 'sulphates': 0.6 , 'volatile_acidity': 0.5, 'total_sulfur_dioxide': 30}
# model.predict(h2o.H2OFrame(pred_dict))

Parse progress: |█████████████████████████████████████████████████████████| 100%
xgboost prediction progress: |████████████████████████████████████████████| 100%


predict
5.87498


In [156]:
def predict(dict_values, model=model):
    y_pred = model.predict(h2o.H2OFrame(pred_dict)).as_data_frame()['predict'][0]
    return y_pred

In [157]:
predict(pred_dict)

Parse progress: |█████████████████████████████████████████████████████████| 100%
xgboost prediction progress: |████████████████████████████████████████████| 100%


5.87497615814209